<a href="https://colab.research.google.com/github/rezlevi/Big-Project/blob/main/testprogram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from sklearn.metrics import mean_squared_error
from statistics import mean
import random
import tensorflow as tf
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
COLS = 3
adatok_nyers=[sor.strip().split(sep=";") for sor in open("/content/drive/My Drive/datas.txt")]
adatok = []
for i in range(1,len(adatok_nyers)):
  adatok.append([float(adatok_nyers[i][1]), float(adatok_nyers[i][2]), float(adatok_nyers[i][3])])

In [ ]:
train=adatok[:20000]
test=adatok[20000:25000]

In [ ]:
def network1(HIST, func1, func2, dropout):
  print('train with tensorflow')
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(HIST, COLS)),
    tf.keras.layers.Dense(100, activation= func1),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Dense(50, activation = 'tanh'),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Dense(COLS, activation= func2)
  ])

  model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

  mselst=[]
  for _ in range(50):
    model.fit(train_x, train_y, epochs=1)
    y=model.predict(test_x)
    print('prediction MSE:', mean_squared_error(test_y, y))
    print('dummy MSE:', mean_squared_error(test_y[HIST:], test_y[HIST-1:-1]))
    mselst.append(mean_squared_error(test_y, y))
  return mselst

In [ ]:
def network2(HIST, func1, func2, dropout):
  print('train with tensorflow')
  model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(units=50, return_sequences=True,input_shape=(HIST,3), activation = func1),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.LSTM(units=50, activation='tanh'),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Dense(COLS, activation=func2)
  ])

  model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

  mselst=[]
  for _ in range(50):
    model.fit(train_x, train_y, epochs=1)
    y=model.predict(test_x)
    print('prediction MSE:', mean_squared_error(test_y, y))
    print('dummy MSE:', mean_squared_error(test_y[HIST:], test_y[HIST-1:-1]))
    mselst.append(mean_squared_error(test_y, y))
  return mselst

In [ ]:
networks = [network1,network2]

In [ ]:
HISTS = [8,10,12]

In [ ]:
functions = ['tanh','sigmoid','relu']

In [ ]:
dropouts = [0.1, 0.2, 0.3]

In [ ]:
mselsts = []
for i in range(len(networks)):
  for j in range(len(HISTS)):
    HIST = HISTS[j]
    import numpy as np
    train_x=(np.array([ train[i:i+HIST] for i in range(len(train)-HIST)])-8)/40
    test_x=(np.array([ test[i:i+HIST] for i in range(len(test)-HIST)])-8)/40
    train_y=(np.array([ train[i+HIST] for i in range(len(train)-HIST)])-8)/40
    test_y=(np.array([ test[i+HIST] for i in range(len(test)-HIST)])-8)/40
    for k in range(len(functions)):
      for l in range(len(functions)):
        for m in range(len(dropouts)):
          mselsts.append([i, HISTS[j], functions[k], functions[l], dropouts[m]],networks[i](HISTS[j],functions[k], functions[l],dropouts[m]))